<a href="https://colab.research.google.com/github/BeomJinJeong/KPUproj/blob/master/cuda10_0_tensorflow_gpu1_13_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow-gpu==1.13.1
!apt-get --purge remove "*cublas*" "cuda*"
!reboot
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt install cuda-10-0
!reboot
!pip install ffmpeg-python
!sudo pip install --upgrade youtube_dl
!pip install flask-ngrok
!pip install pafy

## visualize_cv2.py ▼

In [1]:
import cv2
import numpy as np
import os
import sys
from mrcnn import utils
from mrcnn import model as modellib

os.chdir("/content/drive/My Drive/Colab Notebooks/MRCNN_pure")
sys.path.append("/content/drive/My Drive/Colab Notebooks/MRCNN_pure")
ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco  /"))  # To find local version
import coco

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)


class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceConfig()
config.display()

model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config
)
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = [
    'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush'
]


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image


def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image


if __name__ == '__main__':
    """
        test everything
    """

    capture = cv2.VideoCapture(0)

    # these 2 lines can be removed if you dont have a 1080p camera.
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

    while True:
        ret, frame = capture.read()
        results = model.detect([frame], verbose=0)
        r = results[0]
        frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
        )
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

Using TensorFlow backend.


FileNotFoundError: ignored

### local-web(flask) threading

In [ ]:
import cv2
import os
import sys
sys.path.append("/MRCNN_pure")
from visualize_cv2 import model, display_instances, class_names
import argparse
import threading
from flask import Flask, render_template, Response

capture = cv2.VideoCapture(1)

outFrame = None
isEmergency = "false"
_lock = threading.Lock()

app = Flask(__name__)

@app.route("/")
def main():
  return render_template('index.html')

def webcam(capture):
  global outFrame, _lock, isEmergency

  while True:

    grabbed, frame = capture.read()
    if not grabbed:
      print ("Not grabbed.")

    results = model.detect([frame], verbose=1)
    
    # Visualize results
    r = results[0]
    masked_frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                              class_names, r['scores'])
    
    with _lock:
      outFrame = masked_frame.copy()
      isEmergency = "false"

def generate():

    global outFrame, _lock

    while True:
      with _lock:
        if outFrame is None:
          continue
        
        encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
        (flag, encodedImage) = cv2.imencode(".jpg", outFrame, encode_param)
        if not flag:
          continue
      
      yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')


@app.route("/video_feed")
def video_feed():
  return Response(generate(), mimetype = "multipart/x-mixed-replace; boundary=frame")


def emergency():
  from PIL import Image
  isEme = cv2.imread('test1.jpg')
  isEmerg = True

  encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
  flag, encodedImage = cv2.imencode(".jpg", isEme, encode_param)
  while True:
    with _lock:
      if isEmerg:
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')

@app.route("/isEmg")
def isEmg():
  return Response(emergency(), mimetype = "multipart/x-mixed-replace; boundary=frame")
print('server start')

def log_gen():
  text1 = "sibal"
  print("hi")
  yield text1
    
@app.route("/for_log")
def for_log():
  print("hello")
  return Response(log_gen())

def isEmg_whole():
  global isEmergency
  with _lock:
    print("isEmg")
    yield isEmergency

@app.route("/isEmergence")
def isEmergence():
  return Response(isEmg_whole());

args = sys.argv
if(len(args) <2):
  print("run command: python or video: file name")
  sys.exit(0) 
name = args[1]
if(len(args[1]) == 0):
  name = int(args[1])


if __name__ == '__main__':
  
  threading.Thread(target=app.run).start()
  threading.Thread(target=webcam(capture)).start()

In [2]:
!pip install flask

### Model - Web - Local threading

In [1]:
import cv2
import os
import sys
#sys.path.append("/MRCNN_pure")
#from visualize_cv2 import model, display_instances, class_names
import argparse
import threading
import matplotlib.pyplot as plt
import numpy as np
from flask import Flask, render_template, Response

os.chdir("/MRCNN_pure_02")
sys.path.append("/MRCNN_pure_02")
  
# Root directory of the project
ROOT_DIR = os.path.abspath(".")

MAXIMUM_WEIGHT = 10
layer = 255 * np.ones((500,500,3), dtype=np.uint8)
layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
logo = cv2.imread(os.path.join(ROOT_DIR, "siren_black.jpg"))

from keras.models import load_model
mlp_model = load_model('/MRCNN_pure_02/mlp_model.h5')
  
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
  
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
  
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
  
  
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
  
config = InferenceConfig()
config.display()
  
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
  
# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
  
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def apply_mask2(image, mask, num=2):
    image[:, :] = np.where(
        mask == 1,
        image[:, :] +num,
        image[:, :]
    )
    return image    

def img_overlay(background, logo, height, xpos, ypos):
    logo_height = int(height / 7)
    logo = cv2.resize(logo, (logo_height, logo_height))

    gray_logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
    _, mask_inv = cv2.threshold(gray_logo, 10, 255, cv2.THRESH_BINARY_INV)


    background_height, background_width, _ = background.shape # 400, 520, 3
    logo_height, logo_width, _ = logo.shape # 308, 250, 3

    x = xpos
    y = ypos

    roi = background[x: x+logo_height, y: y+logo_width]

    roi_logo = cv2.add(logo, roi, mask=mask_inv)
    result = cv2.add(roi_logo, logo)
    np.copyto(roi, result)
    return background

def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    os.chdir("/MRCNN_pure")
    n_instances = boxes.shape[0]
    isEmergency = 0
    emergencyImage = image

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue
        if not (ids[i] == 1) :
            continue
        print('test27')
        image = cv2.UMat(image)
        image = cv2.UMat.get(image)
        height = image.shape[0]
        width = image.shape[1]

        mask_list = []
        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]
        mask_list.append(mask)

        global layer
        global layer_frame
        global weight_frame

        black = [0, 0, 0]
        red = [0, 0, 255]

        weight_frame = apply_mask2(weight_frame, mask)
        weight_frame[weight_frame>=MAXIMUM_WEIGHT] = MAXIMUM_WEIGHT


        current_frame = np.zeros((height,width), dtype=np.uint8)
        current_frame = apply_mask2(current_frame, mask, num = MAXIMUM_WEIGHT)
        current_frame[current_frame==0] = 255

        pixel_count = np.count_nonzero(current_frame==MAXIMUM_WEIGHT)
        print("Current frame pixels : ", pixel_count)
        same_pixel = np.count_nonzero(current_frame==weight_frame)
        print("Same pixels: ", same_pixel)
        frame_acc = same_pixel / pixel_count
        print("Frame acc: ", frame_acc)

        layer = 255 * np.ones((height,width,3), dtype=np.uint8)
        layer = apply_mask(layer, mask, black, alpha=0.8)
        layer = layer[y1:y2, x1:x2, :]
        layer_height = layer.shape[0]
        layer_width = layer.shape[1]
        if(layer_height >= layer_width) :
          scale_percent = 64 / layer_height
          new_height = int(layer_height * scale_percent)
          new_width = int(layer_width * scale_percent)
          layer = cv2.resize(layer, dsize=(new_width, new_height), interpolation=cv2.INTER_CUBIC)
          layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
          #layer_frame[0:new_width, 32-(new_height//2):32+(new_height//2),:] = layer
          rows, cols, channels = layer.shape
          layer_frame[0:rows, 32-(cols//2):32-(cols//2) + cols] = layer
        else :
          scale_percent = 64 / layer_width
          new_height = int(layer_height * scale_percent)
          new_width = int(layer_width * scale_percent)
          layer = cv2.resize(layer, dsize=(new_width, new_height), interpolation=cv2.INTER_CUBIC)
          layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
          #layer_frame[32-(new_width//2):32+(new_width//2), 0:new_height,:] = layer
          rows, cols, channels = layer.shape
          layer_frame[32-(rows//2):32-(rows//2) + rows, 0:cols] = layer

        backup = layer_frame
        layer_frame = cv2.cvtColor(layer_frame, cv2.COLOR_BGR2GRAY)
        layer_frame = ~layer_frame

        predict_test = layer_frame
        predict_test = predict_test.reshape(1, 4096)

        result = mlp_model.predict(predict_test)
        label_index = np.argmax(result)

        rows, cols = np.where(weight_frame >= 10)
        print(rows, cols)

        image = apply_mask(image, mask, color)

        contour_frame = np.zeros((height,width), dtype=np.uint8)
        contour_frame = apply_mask2(contour_frame, mask, num=255)
        ret, thr = cv2.threshold(contour_frame, 127, 255, 0)
        contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        
        if frame_acc >= 0.9 and label_index != 1:
          isEmergency = 1

        if (label_index != 1) :
          caption = 'EMergency! {} {:.2f}'.format(label, score) if score else label
          #image = cv2.rectangle(image, (x1, y1), (x2, y2), red, 2)
          if isEmergency == 1:
            emergencyImage = image[y1:y2, x1:x2, :]
            cv2.drawContours(image, contours, -1, (0,0,255), 1)
          else :
            cv2.drawContours(image, contours, -1, (0,255,255), 1)
        else :
          cv2.drawContours(image, contours, -1, (0,255,0), 1)
          #image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )


    print(np.max(weight_frame))
    weight_frame[weight_frame>=1] -= 1
    global logo
    if isEmergency == 1:
      image = img_overlay(image, logo, image.shape[0], 0, 0)

    return image, emergencyImage, isEmergency

capture = cv2.VideoCapture(1)

(grabbed, frame) = capture.read()
image_h = frame.shape[0]
image_w = frame.shape[1]
weight_frame = np.zeros((image_h,image_w), dtype=np.uint8)
_lock = threading.Lock() # 쓰레딩 시 사용되는 lock

frame_for_crop = None # 응급 상황 크롭 이미지 변수
emer_for_bool = 0 # 응급상황 여부 변수. 0이면 정상, 1이면 응급 상황
frame_for_screen = None # 현재 영상 이미지 변수

app = Flask(__name__)

@app.route("/")
def main():
  return render_template('index.html')

# 영상 프레임 처리 후 결과 반환 쓰레드
def webcam(capture):
  global frame_for_screen, _lock, emer_for_bool, frame_for_crop

  while True:

    grabbed, frame = capture.read()
    if not grabbed:
      print ("Not grabbed.")
    
    results = model.detect([frame], verbose=1)
    
    # Visualize results
    r = results[0]
    masked_frame , masked_list, isEmergency = display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                              class_names, r['scores'])
    
    with _lock:
      frame_for_screen = masked_frame.copy()
      frame_for_crop = masked_list.copy()
      emer_for_bool = isEmergency

# 현재 영상의 프레임을 스크린에 띄워주는 제너레이터
def generate():

    global frame_for_screen, _lock

    while True:
      with _lock:
        if frame_for_screen is None:
          continue
        
        encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
        (flag, encodedImage) = cv2.imencode(".jpg", frame_for_screen, encode_param)
        if not flag:
          continue
      
      yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')


@app.route("/video_feed")
def video_feed():
  return Response(generate(), mimetype = "multipart/x-mixed-replace; boundary=frame")


# 응급상황의 크롭 이미지를 띄워주는 제너레이터
def emergency():
  global emer_for_bool, frame_for_crop
  count = 0
  while True:
    with _lock:
      if emer_for_bool==1 and (count==0 or count==1):
        encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
        flag, encodedImage = cv2.imencode(".jpg", frame_for_crop, encode_param)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')
        count+=1
        continue
      elif emer_for_bool==1 and count==1:
        count=2
      elif emer_for_bool==0:
        count=0
  

@app.route("/isEmg")
def isEmg():
  return Response(emergency(), mimetype = "multipart/x-mixed-replace; boundary=frame")
print('server start')

# 응급상황의 여부를 로그로 알려주는 제너레이터
def log_gen():
  global emer_for_bool

  if emer_for_bool==1:
    emer_for_text = "emergency"
    yield emer_for_text
  else :
    emer_for_text = "normal"
    yield emer_for_text
    
@app.route("/for_log")
def for_log():
  return Response(log_gen())

# 경고상황의 여부를 로그로 알려주는 제너레이터
def log_gen_warn():
  global emer_for_bool, weight_frame

  if (emer_for_bool==0) and (np.max(weight_frame)== MAXIMUM_WEIGHT):
    emer_for_text = "warning"
    yield emer_for_text
    
@app.route("/for_warn_log")
def for_warn_log():
  return Response(log_gen_warn())

def log_gen_weight():
  global MAXIMUM_WEIGHT
  yield MAXIMUM_WEIGHT
  
@app.route("/for_warn_weight")
def for_warn_weight():
  return Response(log_gen_weight())

args = sys.argv
if(len(args) <2):
  print("run command: python or video: file name")
  sys.exit(0) 
name = args[1]
if(len(args[1]) == 0):
  name = int(args[1])


if __name__ == '__main__':
# 쓰레딩 시작  
  threading.Thread(target=app.run).start()
  threading.Thread(target=webcam(capture)).start()  

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Processing 1 images
image                    shape: (480, 640, 3)         min:    0.00000  max:  255.00000  uint8


127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 16:33:20] "GET /for_log HTTP/1.1" 200 -


...

127.0.0.1 - - [26/Nov/2020 16:34:08] "GET /for_log HTTP/1.1" 200 -


molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
test27
Current frame pixels :  64736
Same pixels:  29721
Frame acc:  0.4591108502224419
[212 213 213 ... 470 470 470] [197 196 197 ... 433 434 435]
10
Processing 1 images
image                    shape: (480, 640, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\JeongBeomJin\Anaconda3\envs\OpenCV\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-0648f3f8564c>", line 388, in <module>
    threading.Thread(target=webcam(capture)).start()
  File "<ipython-input-1-0648f3f8564c>", line 283, in webcam
    results = model.detect([frame], verbose=1)
  File "C:\MRCNN_pure_02\mrcnn\model.py", line 2524, in detect
    self.keras_model.predict([molded_images, image_metas, anchors], verbose=0)
  File "C:\Users\JeongBeomJin\Anaconda3\envs\OpenCV\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "C:\Users\JeongBeomJin\Anaconda3\envs\OpenCV\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "C:\Users\JeongBeomJin\Anaconda3\envs\OpenCV\lib\site-packages\tensorflow\python\keras\backend.py", line 3

KeyboardInterrupt: ignored

demo_test

In [ ]:
import cv2
import os
import sys
#sys.path.append("/MRCNN_pure")
#from visualize_cv2 import model, display_instances, class_names
import argparse
import threading
import matplotlib.pyplot as plt
import numpy as np
from flask import Flask, render_template, Response

os.chdir("/MRCNN_pure")
sys.path.append("/MRCNN_pure")
  
# Root directory of the project
ROOT_DIR = os.path.abspath(".")

MAXIMUM_WEIGHT = 10
layer = 255 * np.ones((500,500,3), dtype=np.uint8)
layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
logo = cv2.imread(os.path.join(ROOT_DIR, "siren_black.jpg"))

from keras.models import load_model
mlp_model = load_model('/MRCNN_pure/mlp_model.h5')
  
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
  
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
  
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
  
  
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
  
config = InferenceConfig()
config.display()
  
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
  
# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
  
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def apply_mask2(image, mask, num=2):
    image[:, :] = np.where(
        mask == 1,
        image[:, :] +num,
        image[:, :]
    )
    return image    

def img_overlay(background, logo, height, xpos, ypos):
    logo_height = int(height / 7)
    logo = cv2.resize(logo, (logo_height, logo_height))

    gray_logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
    _, mask_inv = cv2.threshold(gray_logo, 10, 255, cv2.THRESH_BINARY_INV)


    background_height, background_width, _ = background.shape # 400, 520, 3
    logo_height, logo_width, _ = logo.shape # 308, 250, 3

    x = xpos
    y = ypos

    roi = background[x: x+logo_height, y: y+logo_width]

    roi_logo = cv2.add(logo, roi, mask=mask_inv)
    result = cv2.add(roi_logo, logo)
    np.copyto(roi, result)
    return background

def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    os.chdir("/MRCNN_pure")
    n_instances = boxes.shape[0]
    isEmergency = 0
    emergencyImage = image

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue
        if not (ids[i] == 1) :
            continue
        print('test27')
        image = cv2.UMat(image)
        image = cv2.UMat.get(image)
        height = image.shape[0]
        width = image.shape[1]

        mask_list = []
        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]
        mask_list.append(mask)

        global layer
        global layer_frame
        global weight_frame

        black = [0, 0, 0]
        red = [0, 0, 255]

        weight_frame = apply_mask2(weight_frame, mask)
        weight_frame[weight_frame>=MAXIMUM_WEIGHT] = MAXIMUM_WEIGHT


        current_frame = np.zeros((height,width), dtype=np.uint8)
        current_frame = apply_mask2(current_frame, mask, num = MAXIMUM_WEIGHT)
        current_frame[current_frame==0] = 255

        pixel_count = np.count_nonzero(current_frame==MAXIMUM_WEIGHT)
        print("Current frame pixels : ", pixel_count)
        same_pixel = np.count_nonzero(current_frame==weight_frame)
        print("Same pixels: ", same_pixel)
        frame_acc = same_pixel / pixel_count
        print("Frame acc: ", frame_acc)

        layer = 255 * np.ones((height,width,3), dtype=np.uint8)
        layer = apply_mask(layer, mask, black, alpha=0.8)
        layer = layer[y1:y2, x1:x2, :]
        layer_height = layer.shape[0]
        layer_width = layer.shape[1]
        if(layer_height >= layer_width) :
          scale_percent = 64 / layer_height
          new_height = int(layer_height * scale_percent)
          new_width = int(layer_width * scale_percent)
          layer = cv2.resize(layer, dsize=(new_width, new_height), interpolation=cv2.INTER_CUBIC)
          layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
          #layer_frame[0:new_width, 32-(new_height//2):32+(new_height//2),:] = layer
          rows, cols, channels = layer.shape
          layer_frame[0:rows, 32-(cols//2):32-(cols//2) + cols] = layer
        else :
          scale_percent = 64 / layer_width
          new_height = int(layer_height * scale_percent)
          new_width = int(layer_width * scale_percent)
          layer = cv2.resize(layer, dsize=(new_width, new_height), interpolation=cv2.INTER_CUBIC)
          layer_frame = 255 * np.ones((64,64,3), dtype=np.uint8)
          #layer_frame[32-(new_width//2):32+(new_width//2), 0:new_height,:] = layer
          rows, cols, channels = layer.shape
          layer_frame[32-(rows//2):32-(rows//2) + rows, 0:cols] = layer

        backup = layer_frame
        layer_frame = cv2.cvtColor(layer_frame, cv2.COLOR_BGR2GRAY)
        layer_frame = ~layer_frame

        predict_test = layer_frame
        predict_test = predict_test.reshape(1, 4096)

        result = mlp_model.predict(predict_test)
        label_index = np.argmax(result)

        rows, cols = np.where(weight_frame >= 10)
        print(rows, cols)

        image = apply_mask(image, mask, color)

        contour_frame = np.zeros((height,width), dtype=np.uint8)
        contour_frame = apply_mask2(contour_frame, mask, num=255)
        ret, thr = cv2.threshold(contour_frame, 127, 255, 0)
        contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        
        if frame_acc >= 0.7 and label_index != 1:
          isEmergency = 1

        if (label_index != 1) :
          caption = 'EMergency! {} {:.2f}'.format(label, score) if score else label
          #image = cv2.rectangle(image, (x1, y1), (x2, y2), red, 2)
          if isEmergency == 1:
            emergencyImage = image[y1:y2, x1:x2, :]
            cv2.drawContours(image, contours, -1, (0,0,255), 1)
          else :
            cv2.drawContours(image, contours, -1, (0,255,255), 1)
        else :
          cv2.drawContours(image, contours, -1, (0,255,0), 1)
          #image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )


    print(np.max(weight_frame))
    weight_frame[weight_frame>=1] -= 1
    global logo
    if isEmergency == 1:
      image = img_overlay(image, logo, image.shape[0], 0, 0)

    return image, emergencyImage, isEmergency

capture = cv2.VideoCapture(os.path.join(ROOT_DIR, "test13.mp4"))

(grabbed, frame) = capture.read()
image_h = frame.shape[0]
image_w = frame.shape[1]
weight_frame = np.zeros((image_h,image_w), dtype=np.uint8)
_lock = threading.Lock() # 쓰레딩 시 사용되는 lock

frame_for_crop = None # 응급 상황 크롭 이미지 변수
emer_for_bool = 0 # 응급상황 여부 변수. 0이면 정상, 1이면 응급 상황
frame_for_screen = None # 현재 영상 이미지 변수

app = Flask(__name__)

@app.route("/")
def main():
  return render_template('index.html')

# 영상 프레임 처리 후 결과 반환 쓰레드
def webcam(capture):
  global frame_for_screen, _lock, emer_for_bool, frame_for_crop

  while True:
    grabbed, frame = capture.read()
    frame_pos = capture.get(cv2.CAP_PROP_POS_FRAMES)
    frame_count = capture.get(cv2.CAP_PROP_FRAME_COUNT)
    if not grabbed:
      print ("Not grabbed.")
    
    if ((frame_pos%25) == 0): 
      results = model.detect([frame], verbose=1)
      
      # Visualize results
      r = results[0]
      masked_frame , masked_list, isEmergency = display_instances(frame, r['rois'], r['masks'], r['class_ids'],
                                class_names, r['scores'])
      
      with _lock:
        frame_for_screen = masked_frame.copy()
        frame_for_crop = masked_list.copy()
        emer_for_bool = isEmergency

# 현재 영상의 프레임을 스크린에 띄워주는 제너레이터
def generate():

    global frame_for_screen, _lock

    while True:
      with _lock:
        if frame_for_screen is None:
          continue
        
        encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
        (flag, encodedImage) = cv2.imencode(".jpg", frame_for_screen, encode_param)
        if not flag:
          continue
      
      yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')


@app.route("/video_feed")
def video_feed():
  return Response(generate(), mimetype = "multipart/x-mixed-replace; boundary=frame")


# 응급상황의 크롭 이미지를 띄워주는 제너레이터
def emergency():
  global emer_for_bool, frame_for_crop
  count = 0
  while True:
    with _lock:
      if emer_for_bool==1 and (count==0 or count==1):
        encode_param=[int(cv2.IMWRITE_JPEG_QUALITY), 90]
        flag, encodedImage = cv2.imencode(".jpg", frame_for_crop, encode_param)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')
        count+=1
        continue
      elif emer_for_bool==1 and count==1:
        count=2
      elif emer_for_bool==0:
        count=0
  

@app.route("/isEmg")
def isEmg():
  return Response(emergency(), mimetype = "multipart/x-mixed-replace; boundary=frame")
print('server start')

# 응급상황의 여부를 로그로 알려주는 제너레이터
def log_gen():
  global emer_for_bool

  if emer_for_bool==1:
    emer_for_text = "emergency"
    yield emer_for_text
  else :
    emer_for_text = "normal"
    yield emer_for_text
    
@app.route("/for_log")
def for_log():
  return Response(log_gen())

# 경고상황의 여부를 로그로 알려주는 제너레이터
def log_gen_warn():
  global emer_for_bool, weight_frame

  if (emer_for_bool==0) and (np.max(weight_frame)== MAXIMUM_WEIGHT):
    emer_for_text = "warning"
    yield emer_for_text
    
@app.route("/for_warn_log")
def for_warn_log():
  return Response(log_gen_warn())

args = sys.argv
if(len(args) <2):
  print("run command: python or video: file name")
  sys.exit(0) 
name = args[1]
if(len(args[1]) == 0):
  name = int(args[1])


if __name__ == '__main__':
# 쓰레딩 시작  
  threading.Thread(target=app.run).start()
  threading.Thread(target=webcam(capture)).start()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Processing 1 images
image                    shape: (480, 852, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


127.0.0.1 - - [25/Nov/2020 16:15:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:12] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:13] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:14] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:15] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:19] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:19] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:19] "GET /for_log HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 16:15:19] "GET /for_log HTTP/1.1" 200 -


...

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


127.0.0.1 - - [25/Nov/2020 16:16:06] "GET /for_log HTTP/1.1" 200 -


test27
Current frame pixels :  1486
Same pixels:  1379
Frame acc:  0.9279946164199192
[163 163 163 ... 201 201 201] [76 77 78 ... 95 96 97]
10
Processing 1 images
image                    shape: (480, 852, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


127.0.0.1 - - [25/Nov/2020 16:16:07] "GET /for_log HTTP/1.1" 200 -


test27
Current frame pixels :  1558
Same pixels:  1339
Frame acc:  0.8594351732991015
[163 163 163 ... 201 201 201] [76 77 78 ... 94 95 96]
10
Processing 1 images
image                    shape: (480, 852, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
test27
Current frame pixels :  1512
Same pixels:  1266
Frame acc:  0.8373015873015873
[163 163 163 ... 198 198 198] [76 77 78 ... 95 96 97]
10
Processing 1 images
image                    shape: (480, 852, 3)         min:    0.00000  max:  255.00000  uint8
